In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, KFold
import lightgbm
import xgboost
import pickle
import joblib
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Concatenate, Input
import re
import optuna
from sklearn.ensemble import VotingClassifier
import json

In [2]:
def prepare_data(file_path):
    df = pd.read_csv(file_path)
    df = df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
    df = df[df.new_last_price_30_std < 0.2]
    df['date_m'] = pd.to_datetime(df['date_m'])
    df = df.sort_values('date_m')
    LE = LabelEncoder()
    cat_features = []
    for c in df.columns:
        if df[c].dtypes == 'object':
            print(c)
            cat_features.append(c)
            df[c] = LE.fit_transform(df[c])
            np.save(f'D:\\Desktop\\Time\\classes_{c}.npy', LE.classes_)  # ذخیره فایل‌ها در آدرس مورد نظر
    df = df[df['date_m'] > '2010-01-01']
    
    return df

In [3]:
def create_target(df, upper_bound=1.07, lower_bound=1.0):
    df['target_'] = -1
    df.loc[df.target_max > upper_bound, 'target_'] = 1
    df.loc[(df.target_max <= upper_bound) & (df.target_end < lower_bound), 'target_'] = 0
    return df

In [4]:
def X_y_split(df):
    val = df.loc[df['date_m'] >= '2022-01-01']
    df_ = df.loc[df['date_m'] < '2022-01-01']
    X = df_.drop(['target_max', 'target_end', 'target_', 'date_m'], axis=1)
    Y = df_['target_']
    X = X[Y != -1]
    Y = Y[Y != -1]
    return X, Y, val

In [5]:
def optimize_hyperparameter(X, Y, model, thresh):
    kf = KFold(n_splits=5, shuffle=False)
    K = 0
    for train_index, test_index in kf.split(X):
        K += 1
        if K <= 4:
            continue
        Xtrain, Xtest = X.iloc[train_index], X.iloc[test_index]
        Ytrain, Ytest = Y.iloc[train_index], Y.iloc[test_index]

    def objective(trial):
        if model == lightgbm.LGBMClassifier:
            param_dist = {
                'learning_rate': trial.suggest_categorical('learning_rate', list(np.arange(0.001, 1, 0.001))),
                'num_leaves': trial.suggest_int('num_leaves', 20, 200),
                'colsample_bytree': trial.suggest_categorical('colsample_bytree', list(np.arange(0.7, 0.95, 0.05))),
                'max_depth': trial.suggest_int('max_depth', 5, 50),
                'random_state': 2020,
                'verbose': 1,
                'early_stopping_rounds': 50,
                'n_estimators': 200
            }
        elif model == xgboost.XGBClassifier:
            param_dist = {
                'learning_rate': trial.suggest_categorical('learning_rate', list(np.arange(0.001, 0.1, 0.001))),
                'colsample_bytree': trial.suggest_categorical('colsample_bytree', list(np.arange(0.7, 0.95, 0.05))),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'random_state': 2020,
                'verbosity': 1
            }

        tr_model = model(**param_dist)
        tr_model.fit(Xtrain, Ytrain, eval_set=[(Xtest, Ytest)])
        pred_proba = tr_model.predict_proba(Xtest)[:, 1]
        roc_score = roc_auc_score(Ytest, pred_proba)

        return roc_score
            
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=30)

    best_params = study.best_trial.params
    best_model = model(**best_params)
    best_model.fit(Xtrain, Ytrain, eval_set=[(Xtest, Ytest)])
    pred = best_model.predict(Xtest)
    pred_proba = best_model.predict_proba(Xtest)[:, 1]
    roc_score = roc_auc_score(Ytest, pred_proba)
    mask = pred_proba > thresh
    pred = np.zeros(len(pred_proba))
    pred[mask] = 1
    print(roc_score)
    print(confusion_matrix(Ytest[mask], pred[mask]))

    return roc_score, best_params


In [6]:
def model_result(X, y, val, xgb_params, lgb_params, weights, thresh):
    models = [('xgboost', xgboost.XGBClassifier(**xgb_params)), ('lightgbm', lightgbm.LGBMClassifier(**lgb_params))]
    #opt_model = VotingClassifier(estimators=models, voting='soft', weights=weights)
    #opt_model.fit(X, y)
    opt_model = lightgbm.LGBMClassifier(**lgb_params)
    opt_model.fit(X, y)
    
    X_date = val['date_m']
    X_val = val.drop(['target_max', 'target_end', 'target_', 'date_m'], axis=1)
    Y_val = val['target_']
    val_t = val[Y_val != -1]
    date_not_middle = X_date[Y_val != -1]
    X_not_middle = X_val[Y_val != -1]
    Y_not_middle = Y_val[Y_val != -1]
    pred_proba_not_middle_lgb = opt_model.predict_proba(X_not_middle)[:, 1]
    pred_not_middle_lgb = opt_model.predict(X_not_middle)
    mask = pred_proba_not_middle_lgb > thresh
    print('val roc:', roc_auc_score(Y_not_middle, pred_proba_not_middle_lgb))
    print('ratio:', len(pred_proba_not_middle_lgb[mask]) / len(pred_proba_not_middle_lgb))
    print('confusion matrix', confusion_matrix(Y_not_middle[mask], pred_not_middle_lgb[mask]))
    print('-----------------------------')

    date_ = X_date[Y_val == -1]
    X_ = X_val[Y_val == -1]
    Y_ = Y_val[Y_val == -1]
    pred_proba_middle_lgb = opt_model.predict_proba(X_)
    pred_middle_lgb = opt_model.predict(X_)
    mask_ = pred_proba_middle_lgb[:, 1] > thresh
    print(f'predict middle class as 1: {len(pred_middle_lgb[mask_])}, predict middle class as 0: {len(pred_middle_lgb[~mask_])}')
    print('ratio:', len(pred_middle_lgb[mask_]) / len(pred_middle_lgb))

    return date_not_middle[mask], X_not_middle[mask], Y_not_middle[mask], val_t, pred_proba_not_middle_lgb, opt_model


In [7]:
file_path = r"D:\Desktop\Time\features_added.csv"
up_bound = 1.07
low_band = 1.0
thresh = 0.68
model = lightgbm.LGBMClassifier
df = prepare_data(file_path)
df = df.drop(columns='Unnamed0', errors='ignore')
df = create_target(df, upper_bound=up_bound, lower_bound=low_band)
X, y, val = X_y_split(df)
roc_lgb, best_lgb = optimize_hyperparameter(X, y, model=model, thresh=thresh)

date_sh
fa_symbol
en_symbol_12_digit_code
sub_industry_code
industry_code
index
en_symbol
en_name
fa_name


[I 2024-11-06 18:22:57,135] A new study created in memory with name: no-name-23e15bb0-7d73-4254-a0a8-22a3f27ab494


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 149285, number of negative: 265026
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.185120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26371
[LightGBM] [Info] Number of data points in the train set: 414311, number of used features: 108
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360321 -> initscore=-0.573971
[LightGBM] [Info] Start training from score -0.573971
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[183]	valid_0's binary_logloss: 0.270504


[I 2024-11-06 18:23:07,526] Trial 0 finished with value: 0.966272559265854 and parameters: {'learning_rate': 0.34800000000000003, 'num_leaves': 24, 'colsample_bytree': 0.75, 'max_depth': 43}. Best is trial 0 with value: 0.966272559265854.


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 149285, number of negative: 265026
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.185974 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26371
[LightGBM] [Info] Number of data points in the train set: 414311, number of used features: 108
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360321 -> initscore=-0.573971
[LightGBM] [Info] Start training from score -0.573971
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[39]	valid_0's binary_logloss: 0.257501


...

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 149285, number of negative: 265026
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.180110 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26371
[LightGBM] [Info] Number of data points in the train set: 414311, number of used features: 108
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360321 -> initscore=-0.573971
[LightGBM] [Info] Start training from score -0.573971
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[28]	valid_0's binary_logloss: 0.297519


[I 2024-11-06 18:27:58,927] Trial 28 finished with value: 0.9630394624804935 and parameters: {'learning_rate': 0.495, 'num_leaves': 129, 'colsample_bytree': 0.75, 'max_depth': 47}. Best is trial 17 with value: 0.9671662675533212.


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 149285, number of negative: 265026
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.168614 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26371
[LightGBM] [Info] Number of data points in the train set: 414311, number of used features: 108
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360321 -> initscore=-0.573971
[LightGBM] [Info] Start training from score -0.573971
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[105]	valid_0's binary_logloss: 0.308957


[I 2024-11-06 18:28:06,981] Trial 29 finished with value: 0.9558779947120222 and parameters: {'learning_rate': 0.28700000000000003, 'num_leaves': 21, 'colsample_bytree': 0.75, 'max_depth': 34}. Best is trial 17 with value: 0.9671662675533212.


[LightGBM] [Info] Number of positive: 149285, number of negative: 265026
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.172028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26367
[LightGBM] [Info] Number of data points in the train set: 414311, number of used features: 108
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360321 -> initscore=-0.573971
[LightGBM] [Info] Start training from score -0.573971
0.9668042190287359
[[    0  6268]
 [    0 37362]]


In [8]:
thresh = 0.7
model = xgboost.XGBClassifier
df = prepare_data(file_path)
df = df.drop(columns='Unnamed0', errors='ignore')
df = create_target(df, upper_bound=up_bound, lower_bound=low_band)
X, y, val = X_y_split(df)
roc_xgb, best_xgb = optimize_hyperparameter(X, y, model=model, thresh=thresh)


date_sh
fa_symbol
en_symbol_12_digit_code
sub_industry_code
industry_code
index
en_symbol
en_name
fa_name


[I 2024-11-06 18:29:09,614] A new study created in memory with name: no-name-92d9420c-7f60-40a4-bed5-1b6bb045eacf


[0]	validation_0-logloss:0.66016
[1]	validation_0-logloss:0.63706
[2]	validation_0-logloss:0.61715
[3]	validation_0-logloss:0.60728
[4]	validation_0-logloss:0.59469
[5]	validation_0-logloss:0.58132
[6]	validation_0-logloss:0.57056
[7]	validation_0-logloss:0.56129
[8]	validation_0-logloss:0.55388
[9]	validation_0-logloss:0.54742
[10]	validation_0-logloss:0.54171
[11]	validation_0-logloss:0.53519
[12]	validation_0-logloss:0.52935
[13]	validation_0-logloss:0.52789
[14]	validation_0-logloss:0.52395
[15]	validation_0-logloss:0.52375
[16]	validation_0-logloss:0.52137
[17]	validation_0-logloss:0.52055
[18]	validation_0-logloss:0.52072
[19]	validation_0-logloss:0.52044
[20]	validation_0-logloss:0.51920
[21]	validation_0-logloss:0.51759
[22]	validation_0-logloss:0.51672
[23]	validation_0-logloss:0.51452
[24]	validation_0-logloss:0.51255
[25]	validation_0-logloss:0.51122
[26]	validation_0-logloss:0.51050
[27]	validation_0-logloss:0.51051
[28]	validation_0-logloss:0.51155
[29]	validation_0-loglos

[I 2024-11-06 18:29:33,968] Trial 0 finished with value: 0.8777948534735777 and parameters: {'learning_rate': 0.07100000000000001, 'colsample_bytree': 0.8, 'max_depth': 8}. Best is trial 0 with value: 0.8777948534735777.


[0]	validation_0-logloss:0.65714
[1]	validation_0-logloss:0.63030
[2]	validation_0-logloss:0.60837
[3]	validation_0-logloss:0.59673
[4]	validation_0-logloss:0.58358
[5]	validation_0-logloss:0.56905
[6]	validation_0-logloss:0.55738
[7]	validation_0-logloss:0.54930
[8]	validation_0-logloss:0.54164
[9]	validation_0-logloss:0.53502
[10]	validation_0-logloss:0.52975
[11]	validation_0-logloss:0.52372
[12]	validation_0-logloss:0.51940
[13]	validation_0-logloss:0.51706
[14]	validation_0-logloss:0.51498
[15]	validation_0-logloss:0.51611
[16]	validation_0-logloss:0.51404
[17]	validation_0-logloss:0.51530
[18]	validation_0-logloss:0.51453
[19]	validation_0-logloss:0.51470
[20]	validation_0-logloss:0.51440
[21]	validation_0-logloss:0.51319
[22]	validation_0-logloss:0.51266
[23]	validation_0-logloss:0.51082
[24]	validation_0-logloss:0.50995
[25]	validation_0-logloss:0.51013
[26]	validation_0-logloss:0.50984
[27]	validation_0-logloss:0.50917
[28]	validation_0-logloss:0.50904
[29]	validation_0-loglos

...

[0]	validation_0-logloss:0.64414
[1]	validation_0-logloss:0.61709
[2]	validation_0-logloss:0.59735
[3]	validation_0-logloss:0.58744
[4]	validation_0-logloss:0.57345
[5]	validation_0-logloss:0.55934
[6]	validation_0-logloss:0.54904
[7]	validation_0-logloss:0.54098
[8]	validation_0-logloss:0.53416
[9]	validation_0-logloss:0.52954
[10]	validation_0-logloss:0.52483
[11]	validation_0-logloss:0.51931
[12]	validation_0-logloss:0.51802
[13]	validation_0-logloss:0.51636
[14]	validation_0-logloss:0.51518
[15]	validation_0-logloss:0.51429
[16]	validation_0-logloss:0.51258
[17]	validation_0-logloss:0.51468
[18]	validation_0-logloss:0.51472
[19]	validation_0-logloss:0.51479
[20]	validation_0-logloss:0.51418
[21]	validation_0-logloss:0.51529
[22]	validation_0-logloss:0.51483
[23]	validation_0-logloss:0.51370
[24]	validation_0-logloss:0.51335
[25]	validation_0-logloss:0.51384
[26]	validation_0-logloss:0.51335
[27]	validation_0-logloss:0.51267
[28]	validation_0-logloss:0.51341
[29]	validation_0-loglos

[I 2024-11-06 18:43:45,587] Trial 28 finished with value: 0.913205901132524 and parameters: {'learning_rate': 0.094, 'colsample_bytree': 0.9000000000000001, 'max_depth': 10}. Best is trial 10 with value: 0.9258704242682705.


[0]	validation_0-logloss:0.65607
[1]	validation_0-logloss:0.63819
[2]	validation_0-logloss:0.62195
[3]	validation_0-logloss:0.61147
[4]	validation_0-logloss:0.59794
[5]	validation_0-logloss:0.58588
[6]	validation_0-logloss:0.57576
[7]	validation_0-logloss:0.56747
[8]	validation_0-logloss:0.55984
[9]	validation_0-logloss:0.55311
[10]	validation_0-logloss:0.54591
[11]	validation_0-logloss:0.53917
[12]	validation_0-logloss:0.53421
[13]	validation_0-logloss:0.52942
[14]	validation_0-logloss:0.52489
[15]	validation_0-logloss:0.52164
[16]	validation_0-logloss:0.51848
[17]	validation_0-logloss:0.51613
[18]	validation_0-logloss:0.51429
[19]	validation_0-logloss:0.51309
[20]	validation_0-logloss:0.51095
[21]	validation_0-logloss:0.50975
[22]	validation_0-logloss:0.50800
[23]	validation_0-logloss:0.50583
[24]	validation_0-logloss:0.50540
[25]	validation_0-logloss:0.50473
[26]	validation_0-logloss:0.50367
[27]	validation_0-logloss:0.50486
[28]	validation_0-logloss:0.50481
[29]	validation_0-loglos

[I 2024-11-06 18:44:07,008] Trial 29 finished with value: 0.8579956909456146 and parameters: {'learning_rate': 0.061, 'colsample_bytree': 0.9000000000000001, 'max_depth': 7}. Best is trial 10 with value: 0.9258704242682705.


[0]	validation_0-logloss:0.64673
[1]	validation_0-logloss:0.62168
[2]	validation_0-logloss:0.60115
[3]	validation_0-logloss:0.59147
[4]	validation_0-logloss:0.57704
[5]	validation_0-logloss:0.56292
[6]	validation_0-logloss:0.55383
[7]	validation_0-logloss:0.54659
[8]	validation_0-logloss:0.53946
[9]	validation_0-logloss:0.53440
[10]	validation_0-logloss:0.52996
[11]	validation_0-logloss:0.52448
[12]	validation_0-logloss:0.52392
[13]	validation_0-logloss:0.52114
[14]	validation_0-logloss:0.52025
[15]	validation_0-logloss:0.51850
[16]	validation_0-logloss:0.51651
[17]	validation_0-logloss:0.51825
[18]	validation_0-logloss:0.51755
[19]	validation_0-logloss:0.51756
[20]	validation_0-logloss:0.51767
[21]	validation_0-logloss:0.51794
[22]	validation_0-logloss:0.51811
[23]	validation_0-logloss:0.51603
[24]	validation_0-logloss:0.51705
[25]	validation_0-logloss:0.51698
[26]	validation_0-logloss:0.51632
[27]	validation_0-logloss:0.51548
[28]	validation_0-logloss:0.51561
[29]	validation_0-loglos

In [9]:
thresh1 = 0.65
weights = [0.4, 0.6]
X_date, X_1, y_1, val_t, pred_proba_not_middle_lgb, opt_model = model_result(X, y, val, xgb_params=best_xgb, lgb_params=best_lgb, weights=weights, thresh=thresh1)


[LightGBM] [Info] Number of positive: 190916, number of negative: 326972
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.199348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26407
[LightGBM] [Info] Number of data points in the train set: 517888, number of used features: 108
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.368643 -> initscore=-0.538041
[LightGBM] [Info] Start training from score -0.538041
val roc: 0.9898637894933477
ratio: 0.2549609211155943
confusion matrix [[    0   830]
 [    0 42328]]
-----------------------------
predict middle class as 1: 13711, predict middle class as 0: 66751
ratio: 0.1704034202480674


In [10]:
joblib.dump(opt_model, r'D:\Desktop\Time\model.pkl')


['D:\\Desktop\\Time\\model.pkl']

In [11]:
with open(r'D:\Desktop\Time\xgb_hyperparameter.txt', 'w') as xgb_file:
    xgb_file.write(json.dumps(best_xgb))
with open(r'D:\Desktop\Time\lgbm_hyperparameter.txt', 'w') as lgbm_file:
    lgbm_file.write(json.dumps(best_lgb))

In [13]:
f_df= pd.concat([val_t[['target_end', 'target_max']].reset_index()[['target_end', 'target_max']], pd.DataFrame(pred_proba_not_middle_lgb)], axis= 1)
f_df= f_df.rename(columns= {0: 'pred'})
f_df['pred'][f_df['pred'] > 0.65] = 1

In [14]:
f_df= f_df[f_df['pred'] == 1]
num_of_loss= f_df[f_df['target_max'] <= 1.07]['target_end']
num_of_profit= f_df[f_df['target_max'] > 1.07]['target_end']
num_of_loss

48        0.974820
407       0.989130
580       0.955593
776       0.991239
1161      0.983421
            ...   
168615    0.976428
168622    0.977273
168628    0.990855
168654    0.988662
168889    0.999526
Name: target_end, Length: 830, dtype: float64

In [15]:
print(f'profit of this sterategy',(0.07) * len(num_of_profit))

profit of this sterategy 2962.9600000000005


In [16]:
print('loss of this sterategy',(num_of_loss.mean() - 1) * len(num_of_loss))

loss of this sterategy -18.33832909434067


In [17]:
best_xgb

{'learning_rate': 0.093,
 'colsample_bytree': 0.9000000000000001,
 'max_depth': 12}